# Project 1: The Tokenizer Smith - SOLUTION
## Building Byte-Pair Encoding from Scratch

**This notebook contains complete solutions to all tasks.**

## Part 1: Setup and Data Preparation

In [ ]:
import re
from collections import defaultdict, Counter
from typing import List, Dict, Tuple
import matplotlib.pyplot as plt
import numpy as np

# Sample corpus for training
SAMPLE_TEXT = """
The quick brown fox jumps over the lazy dog.
The dog was not amused by the fox's behavior.
Jumping quickly, the fox escaped into the forest.
"""

print(f"Corpus length: {len(SAMPLE_TEXT)} characters")
print(f"Unique characters: {len(set(SAMPLE_TEXT))}")

## Part 2: Implement BPE Core Functions

### Task 1: Get Pair Frequencies - SOLUTION

In [ ]:
def get_pair_frequencies(tokens: List[str]) -> Dict[Tuple[str, str], int]:
    """
    Count the frequency of all adjacent token pairs.
    
    Args:
        tokens: List of current tokens
        
    Returns:
        Dictionary mapping (token1, token2) -> frequency
    """
    pairs = defaultdict(int)
    
    # Iterate through adjacent pairs
    for i in range(len(tokens) - 1):
        pair = (tokens[i], tokens[i + 1])
        pairs[pair] += 1
    
    return pairs

# Test the function
test_tokens = ['h', 'e', 'l', 'l', 'o']
result = get_pair_frequencies(test_tokens)
print("Test input:", test_tokens)
print("Pair frequencies:", dict(result))
print("Expected: {('h', 'e'): 1, ('e', 'l'): 1, ('l', 'l'): 1, ('l', 'o'): 1}")

### Task 2: Merge a Pair - SOLUTION

In [ ]:
def merge_pair(tokens: List[str], pair: Tuple[str, str], new_token: str) -> List[str]:
    """
    Replace all occurrences of a pair with a new merged token.
    
    Args:
        tokens: Current token list
        pair: The pair to merge (token1, token2)
        new_token: The merged token (token1 + token2)
        
    Returns:
        New token list with pairs merged
    """
    new_tokens = []
    i = 0
    
    while i < len(tokens):
        # Check if current position matches the pair
        if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == pair:
            new_tokens.append(new_token)
            i += 2  # Skip both tokens in the pair
        else:
            new_tokens.append(tokens[i])
            i += 1
    
    return new_tokens

# Test the function
test_tokens = ['h', 'e', 'l', 'l', 'o']
result = merge_pair(test_tokens, ('l', 'l'), 'll')
print("Test input:", test_tokens)
print("After merging ('l', 'l') -> 'll':", result)
print("Expected: ['h', 'e', 'll', 'o']")

### Task 3: Main BPE Training Loop - SOLUTION

In [ ]:
def train_bpe(text: str, vocab_size: int, verbose: bool = True) -> Tuple[List[Tuple[str, str]], Dict[str, int]]:
    """
    Train a BPE tokenizer.
    
    Args:
        text: Training corpus
        vocab_size: Target vocabulary size
        verbose: Print merge operations
        
    Returns:
        - List of merge operations in order
        - Final vocabulary with token frequencies
    """
    # Step 1: Initialize with character-level tokens
    tokens = list(text)
    vocab = Counter(tokens)
    merges = []
    
    initial_vocab_size = len(vocab)
    
    # Step 2: Perform merges until vocab_size is reached
    while len(vocab) < vocab_size:
        # 2a. Get pair frequencies
        pairs = get_pair_frequencies(tokens)
        
        if not pairs:
            break
        
        # 2b. Find most frequent pair
        most_frequent_pair = max(pairs.items(), key=lambda x: x[1])
        pair, freq = most_frequent_pair
        
        # 2c. Create new token
        new_token = pair[0] + pair[1]
        
        if verbose:
            print(f"Merge #{len(merges) + 1}: {pair} -> '{new_token}' (freq: {freq})")
        
        # 2d. Merge the pair in token list
        tokens = merge_pair(tokens, pair, new_token)
        
        # 2e. Update vocabulary
        vocab[new_token] = freq
        
        # 2f. Record the merge
        merges.append(pair)
    
    if verbose:
        print(f"\nInitial vocabulary size: {initial_vocab_size}")
        print(f"Final vocabulary size: {len(vocab)}")
        print(f"Number of merges: {len(merges)}")
    
    return merges, vocab

# Train on sample text
merges, vocab = train_bpe(SAMPLE_TEXT, vocab_size=300, verbose=True)
print(f"\nTop 10 tokens by frequency:")
for token, freq in sorted(vocab.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"  '{token}': {freq}")

### Task 4: Encode Text with Learned Merges - SOLUTION

In [ ]:
def encode(text: str, merges: List[Tuple[str, str]]) -> List[str]:
    """
    Tokenize text using learned BPE merges.
    
    Args:
        text: Input text to tokenize
        merges: Ordered list of merge operations
        
    Returns:
        List of tokens
    """
    # Start with character-level tokens
    tokens = list(text)
    
    # Apply each merge in order
    for pair in merges:
        new_token = pair[0] + pair[1]
        tokens = merge_pair(tokens, pair, new_token)
    
    return tokens

# Test encoding
test_sentence = "The quick fox jumps"
encoded = encode(test_sentence, merges)
print(f"Original: {test_sentence}")
print(f"Tokens: {encoded}")
print(f"Compression: {len(test_sentence)} chars -> {len(encoded)} tokens")
print(f"Compression ratio: {len(test_sentence) / len(encoded):.2f}x")

## Part 3: Visualization

### Task 5: Color-Coded Token Visualizer - SOLUTION

In [ ]:
def visualize_tokens(text: str, tokens: List[str]):
    """
    Create a color-coded visualization of tokens.
    """
    # Generate distinct colors for each token
    colors = plt.cm.tab20(np.linspace(0, 1, len(tokens)))
    
    fig, ax = plt.subplots(figsize=(15, 3))
    x_pos = 0
    
    for i, token in enumerate(tokens):
        # Plot each token as a colored rectangle
        width = len(token)
        ax.barh(0, width, left=x_pos, color=colors[i], edgecolor='black', height=0.5)
        
        # Add token text
        # Replace special characters for display
        display_token = token.replace('\n', '\\n').replace(' ', '␣')
        ax.text(x_pos + width/2, 0, display_token, 
               ha='center', va='center', fontsize=8, weight='bold')
        
        x_pos += width
    
    ax.set_xlim(0, x_pos)
    ax.set_ylim(-0.5, 0.5)
    ax.axis('off')
    ax.set_title(f'Token Visualization: "{text}" -> {len(tokens)} tokens', fontsize=12)
    plt.tight_layout()
    plt.show()

# Visualize the encoded sentence
visualize_tokens(test_sentence, encoded)

## Part 4: Ablation Study

### Task 6: Cross-Language Tokenization - SOLUTION

In [ ]:
# Chinese text sample
CHINESE_TEXT = "你好世界。这是一个测试。机器学习很有趣。"

def measure_efficiency(text: str, tokens: List[str]) -> Dict[str, float]:
    """
    Measure tokenization efficiency metrics.
    """
    metrics = {
        'compression_ratio': len(text) / len(tokens),
        'avg_token_length': sum(len(t) for t in tokens) / len(tokens),
        'single_char_tokens': sum(1 for t in tokens if len(t) == 1) / len(tokens) * 100
    }
    return metrics

# Encode Chinese text with English-trained tokenizer
chinese_tokens_english = encode(CHINESE_TEXT, merges)

print("="*60)
print("English tokenizer on Chinese text:")
print("="*60)
print(f"Text: {CHINESE_TEXT}")
print(f"Tokens: {chinese_tokens_english[:20]}...")  # Show first 20
print(f"Total tokens: {len(chinese_tokens_english)}")
metrics_eng = measure_efficiency(CHINESE_TEXT, chinese_tokens_english)
print(f"\nMetrics:")
for key, value in metrics_eng.items():
    print(f"  {key}: {value:.2f}")

# Train a new tokenizer on Chinese
print("\n" + "="*60)
print("Training tokenizer on Chinese text...")
print("="*60)
chinese_merges, chinese_vocab = train_bpe(CHINESE_TEXT, vocab_size=300, verbose=False)
chinese_tokens_native = encode(CHINESE_TEXT, chinese_merges)

print("\nChinese tokenizer on Chinese text:")
print(f"Text: {CHINESE_TEXT}")
print(f"Tokens: {chinese_tokens_native}")
print(f"Total tokens: {len(chinese_tokens_native)}")
metrics_chi = measure_efficiency(CHINESE_TEXT, chinese_tokens_native)
print(f"\nMetrics:")
for key, value in metrics_chi.items():
    print(f"  {key}: {value:.2f}")

# Comparison
print("\n" + "="*60)
print("COMPARISON")
print("="*60)
print(f"Token count - English tokenizer: {len(chinese_tokens_english)}")
print(f"Token count - Chinese tokenizer: {len(chinese_tokens_native)}")
print(f"Efficiency improvement: {len(chinese_tokens_english) / len(chinese_tokens_native):.2f}x")

## Part 5: Analysis and Conclusions

### Questions and Answers:

#### 1. What patterns did you observe in the merge order?

**Answer:** 
- Common character pairs merge first (e.g., 'th', 'he', 'in')
- Frequent words become single tokens (e.g., 'the', 'and')
- Space + common letter combinations merge early
- The algorithm is purely statistical - it has no linguistic knowledge

#### 2. How did the English tokenizer perform on Chinese text?

**Answer:**
- Very poorly! Each Chinese character remained a separate token
- High single-character token percentage (~100%)
- Low compression ratio (close to 1:1)
- This demonstrates why multilingual models need much larger vocabularies

#### 3. What is the relationship between vocabulary size and compression ratio?

**Answer:**
- Larger vocabulary → more merges → longer tokens → better compression
- Trade-off: Larger vocab = larger embedding matrix = more parameters
- Diminishing returns after a certain point
- Typical LLMs use 32k-100k vocab size as a balance

#### 4. Why do multilingual models like mBERT need 100k+ vocabulary size?

**Answer:**
- Each language has unique character combinations
- Need to efficiently represent characters from multiple scripts (Latin, Cyrillic, Chinese, Arabic, etc.)
- Without large vocab, some languages get over-segmented (inefficient)
- 100k+ allows reasonable compression across many languages simultaneously

## Part 6: Additional Visualizations

In [ ]:
# Plot vocabulary growth
def plot_vocab_growth():
    """Track vocabulary size over merge iterations."""
    sizes = [50, 100, 150, 200, 250, 300]
    results = []
    
    for size in sizes:
        merges_temp, vocab_temp = train_bpe(SAMPLE_TEXT, vocab_size=size, verbose=False)
        test_encoded = encode(test_sentence, merges_temp)
        results.append({
            'vocab_size': len(vocab_temp),
            'num_tokens': len(test_encoded),
            'compression': len(test_sentence) / len(test_encoded)
        })
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Vocabulary size vs target
    ax1.plot(sizes, [r['vocab_size'] for r in results], 'o-')
    ax1.set_xlabel('Target Vocabulary Size')
    ax1.set_ylabel('Actual Vocabulary Size')
    ax1.set_title('Vocabulary Growth')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Compression ratio
    ax2.plot(sizes, [r['compression'] for r in results], 'o-', color='green')
    ax2.set_xlabel('Vocabulary Size')
    ax2.set_ylabel('Compression Ratio')
    ax2.set_title('Compression vs Vocabulary Size')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_vocab_growth()

## 🎯 Completion Checklist

- ✅ Implemented `get_pair_frequencies()`
- ✅ Implemented `merge_pair()`
- ✅ Implemented `train_bpe()`
- ✅ Implemented `encode()`
- ✅ Created token visualization
- ✅ Performed cross-language ablation study
- ✅ Analyzed efficiency metrics
- ✅ Answered reflection questions

## Key Takeaways

1. **BPE is purely statistical** - No linguistic knowledge, just frequency
2. **Language-specific tokenizers are more efficient** - Trained on target language
3. **Vocabulary size is a trade-off** - Compression vs model size
4. **Multilingual = larger vocab** - Need to cover many character combinations

## 🚀 Next Project
Move to **02_meaning_space** to learn how these discrete tokens become continuous vector representations!